# Environment

In [43]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb, Tileset
import clodius
import os
import os.path as path
import pandas as pd

import negspy.coordinates as nc

print(os.getcwd())
print(clodius.__version__)
print(higlass.__version__)

/Users/hawleyj/Documents/Davos/Results/2019-10-24_higlass
0.14.2
0.4.0


## Load annotations

In [39]:
# load annotations
chrom_labels = {
    "top": Track(
        track_type="horizontal-chromosome-labels",
        server="http://higlass.io/api/v1",
        tileset_uuid="NyITQvZsS_mOFNlz5C2LJg",
        position="top",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-chromosome-labels",
        server="http://higlass.io/api/v1",
        tileset_uuid="NyITQvZsS_mOFNlz5C2LJg",
        position="left",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    )
}

gene_annots = {
    "top": Track(
        track_type="horizontal-gene-annotations",
        server="http://higlass.io/api/v1",
        tileset_uuid="P0PLbQMwTYGy-5uPIQid7A",
        position="top",
        height=150,
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-gene-annotations",
        server="http://higlass.io/api/v1",
        tileset_uuid="P0PLbQMwTYGy-5uPIQid7A",
        position="left",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    )
}

ctcf_motifs = {
    "top": Track(
        track_type="bedlike",
        server="http://higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="top",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="bedlike",
        server="http://higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="left",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    )
}

hg38 = nc.get_chrominfo("hg38")

# Data
## Contact matrices

In [40]:
metadata = pd.read_csv(path.join("..", "..", "Data", "External", "LowC_Samples_Data_Available.tsv"), sep="\t", header=0)
samples = ["PCa" + str(i) for i in metadata["Sample ID"].tolist()]

benign_metadata = pd.read_csv(path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "config.tsv"), sep="\t", header=0)
benign_metadata = benign_metadata.loc[~benign_metadata["Sample"].str.startswith("PCa"), :]
benign_samples = benign_metadata["Sample"].tolist()

all_samples = samples + benign_samples

cooler_files = (
    [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in samples]
    + [path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "Contacts", s + ".mcool") for s in benign_samples]
)
lowc_tilesets = {s: cooler(f) for s, f in zip(all_samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

lowc_heatmaps = {
    s: {
        "top": Track(
            track_type='horizontal-heatmap',
            position='top',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            },
        ),
        "bottom": Track(
            track_type='horizontal-heatmap',
            position='bottom',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        ),
        "left": Track(
            track_type='vertical-heatmap',
            position='left',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        ),
        "right": Track(
            track_type='vertical-heatmap',
            position='right',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        ),
        "centre": Track(
            track_type='heatmap',
            position='center',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=500,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        )
    } for s in all_samples
}

## TADs

In [48]:
windows = list(range(3, 31))

tad_files = {s: {w: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "separated-TADs", s + ".40000bp.w_" + str(w) + ".domains.beddb") for w in windows} for s in samples}
htad_files = {s: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bed2ddb") for s in samples}
tad_tilesets = {s: {w: beddb(v[w], name=s + " TADs (w=" + str(w) +")") for w in windows} for s, v in tad_files.items()}

def bed2ddb(filepath, uuid=None, **kwargs):
    from clodius.tiles.format import format_dense_tile
    from clodius.tiles.utils import tiles_wrapper_2d
    from clodius.tiles.bed2ddb import get_2d_tileset_info, get_2D_tiles
    return Tileset(
        uuid=uuid,
        tileset_info=lambda: get_2d_tileset_info(filepath),
        tiles=lambda tids: tiles_wrapper_2d(
            tids,
            lambda z,x,y: format_dense_tile(tile_data(z, x, y))
        )
    )

htad_tilesets = {s: bed2ddb(v, name=s + " TADs") for s, v in htad_files.items()}

tads = {
    s: {
        w: {
           "top": Track(
                track_type="bedlike",
                position='top',
                tileset=tad_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "bottom": Track(
                track_type="bedlike",
                position='bottom',
                tileset=tad_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "left": Track(
                track_type="bedlike",
                position="left",
                tileset=tad_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "right": Track(
                track_type="bedlike",
                position="right",
                tileset=tad_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
        } for w in windows
    } for s in samples
}

hierarchical_tads = {
    s: Track(
        track_type="2d-rectangle-domains",
        position="center",
        tileset=htad_tilesets[s],
        filetype="bed2ddb",
    ) for s in samples
}

# Images
## FOXA1

In [9]:
foxa1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"], lowc_heatmaps["Benign-Prostate-1595983"]["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 36000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 39000000, hg38)
    ],
)

display, server, viewconf = higlass.display([foxa1_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'EPrQMCGbQbSu4x0AkoE0BA', 'tracks': {'top': [{'ty…

## _TMPRSS2_-_ERG_ fusion

In [14]:
t2e_view = View(
    tracks=(
        [chrom_labels["top"], gene_annots["top"]]
        + [lowc_heatmaps["PCa" + str(s)]["top"] for s in metadata.loc[metadata["T2E Status"] == "Yes", "Sample ID"]]
        + [lowc_heatmaps["PCa" + str(s)]["top"] for s in metadata.loc[metadata["T2E Status"] == "No", "Sample ID"]]
        + [lowc_heatmaps[s]["top"] for s in benign_samples]
    ),
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr21", 30000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 48000000, hg38)
    ],
)

display, server, viewconf = higlass.display([
    t2e_view
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'MxiCW6RXQpm9iGNIX_e2Ug', 'tracks': {'top': [{'ty…

## T2E translocation

In [13]:
t2e_translocation = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["bottom"],
        lowc_heatmaps["PCa13848"]["centre"],
        lowc_heatmaps["PCa13848"]["left"],
        #hierarchical_tads["PCa13848"]
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)
nont2e_translocation = View(
    tracks=[
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["top"],
        lowc_heatmaps["PCa3023"]["centre"],
        lowc_heatmaps["PCa3023"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)

t2e_translocation_insertion = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["centre"],
        tads["PCa13848"][3]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 34000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37000000, hg38)
    ],
)
nont2e_translocation_insertion = View(
    tracks=[
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["centre"],
        tads["PCa3023"][3]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 34000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37000000, hg38)
    ],
)


display, server, viewconf = higlass.display([t2e_translocation, nont2e_translocation])
#display, server, viewconf = higlass.display([t2e_translocation_insertion, nont2e_translocation_insertion])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'L7pnRKiBRx2CDPApzDAZ0g', 'tracks': {'top': [{'ty…

## AR

In [43]:
ar_tracks = [chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples]
ar_view = View(
    tracks=ar_tracks,
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 65040000, hg38),
        nc.chr_pos_to_genome_pos("chrX", 70200000, hg38)
    ]
)

display, server, viewconf = higlass.display([ar_view], value_scale_syncs = [ar_tracks])
display
#help(higlass.display)

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Cdg-ozpSQUyoZ-v5Z3ibug', 'tracks': {'top': [{'ty…

## MYC

In [30]:
myc_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr8", 126000000, hg38),
        nc.chr_pos_to_genome_pos("chr8", 130000000, hg38)
    ],
)

display, server, viewconf = higlass.display([myc_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'NmT5xaDFQcuUT2CCTwDu2g', 'tracks': {'top': [{'ty…

## CTCF, YY1, HOXB13

In [ ]:
ctcf_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr16", 65000000, hg38),
        nc.chr_pos_to_genome_pos("chr16", 69000000, hg38)
    ],
)
yy1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 98000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 102000000, hg38)
    ],
)
hoxb13_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr17", 46160000, hg38),
        nc.chr_pos_to_genome_pos("chr17", 51320000, hg38)
    ],
)

myc_insertion_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr12", 70480000 - 5e6, hg38),
        nc.chr_pos_to_genome_pos("chr12", 70570000 + 5e6, hg38)
    ],
)

display, server, viewconf = higlass.display([
    myc_insertion_view
])
display

## Miscellaneous

In [15]:
test_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr6", 42500000, hg38),
        nc.chr_pos_to_genome_pos("chr6", 127500000, hg38)
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr6", 42500000, hg38),
        nc.chr_pos_to_genome_pos("chr6", 127500000, hg38)
    ],
)

display, server, viewconf = higlass.display([test_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Ejpo1oruT0m85WNlcl5gZQ', 'tracks': {'top': [{'ty…

In [49]:
display, server, viewconf = higlass.display([
    View(
        tracks=([
            chrom_labels["top"], gene_annots["top"], chrom_labels["left"], gene_annots["left"],
            #lowc_heatmaps["PCa57054"]["centre"],
            tads["PCa57054"][3]["top"],
            tads["PCa57054"][30]["top"],
            hierarchical_tads["PCa57054"],
        ]),
#        initialYDomain=[
#            nc.chr_pos_to_genome_pos("chr21", 30000000, hg38),
#            nc.chr_pos_to_genome_pos("chr21", 48000000, hg38)
#        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr12", 118000000, hg38),
            nc.chr_pos_to_genome_pos("chr12", 135000000, hg38)
        ],
    )
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'I1JaWQ7xQUimdoAkfqVMww', 'tracks': {'top': [{'ty…